In [1]:
import tensorflow.keras as keras

INPUT_SHAPE = [32, 32, 3]
N_OUTPUT_CLASSES = 10

def create_dnn_model(n_hidden=20, n_neurons=100, activation="elu", kernel_initializer="he_normal", lr=0.001):
    """
    Creates a dense neural network
    """

    model = keras.models.Sequential()
    
    # input layer
    model.add(keras.layers.Flatten(input_shape=INPUT_SHAPE))
    
    # add hidden layers
    for _ in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation=activation, kernel_initializer=kernel_initializer))
    
    # output layer
    model.add(keras.layers.Dense(N_OUTPUT_CLASSES, activation="softmax"))

    # configure the model for training
    model.compile(loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"],
                  optimizer=keras.optimizers.Nadam(learning_rate=lr))

    return model

In [2]:
from tensorflow.keras.datasets import cifar10

# load the data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [3]:
from sklearn.model_selection import train_test_split

# create a validation set
X_train_small, X_val, y_train_small, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [4]:
import os
import time

LOG_ROOT = 'train_logs'

def generate_log_dir():
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(LOG_ROOT, run_id)

In [6]:
# create and train a basic DNN model
model = create_dnn_model(lr=5e-5)

model_filepath = "models/dnn.h5"

callbacks = []
callbacks.append(keras.callbacks.EarlyStopping(patience=20))
callbacks.append(keras.callbacks.TensorBoard(log_dir=generate_log_dir()))
callbacks.append(keras.callbacks.ModelCheckpoint(model_filepath, save_best_only=True))

model.fit(X_train_small, y_train_small, validation_data=(X_val, y_val), epochs=1000, callbacks=callbacks, verbose=2)

Train on 40000 samples, validate on 10000 samples
Epoch 1/1000
40000/40000 - 11s - loss: 5.6913 - accuracy: 0.1673 - val_loss: 2.1734 - val_accuracy: 0.2187
Epoch 2/1000
40000/40000 - 9s - loss: 2.0849 - accuracy: 0.2384 - val_loss: 2.0462 - val_accuracy: 0.2477
Epoch 3/1000
40000/40000 - 9s - loss: 1.9735 - accuracy: 0.2787 - val_loss: 1.9881 - val_accuracy: 0.2753
Epoch 4/1000
40000/40000 - 9s - loss: 1.8931 - accuracy: 0.3085 - val_loss: 1.8513 - val_accuracy: 0.3300
Epoch 5/1000
40000/40000 - 9s - loss: 1.8316 - accuracy: 0.3359 - val_loss: 1.8289 - val_accuracy: 0.3349
Epoch 6/1000
40000/40000 - 9s - loss: 1.7810 - accuracy: 0.3535 - val_loss: 1.8002 - val_accuracy: 0.3568
Epoch 7/1000
40000/40000 - 9s - loss: 1.7429 - accuracy: 0.3696 - val_loss: 1.7504 - val_accuracy: 0.3682
Epoch 8/1000
40000/40000 - 10s - loss: 1.6970 - accuracy: 0.3867 - val_loss: 1.7966 - val_accuracy: 0.3546
Epoch 9/1000
40000/40000 - 9s - loss: 1.6612 - accuracy: 0.4005 - val_loss: 1.6890 - val_accuracy: 0

In [7]:
model = keras.models.load_model("models/dnn.h5")
model.evaluate(X_val, y_val)

10000/10000 [==============================] - 1s 70us/sample - loss: 1.5296 - accuracy: 0.4606


[1.52964987449646, 0.4606]

The model with the lowest validation loss reaches 46.1% validation accuracy. It took 49 epochs to reach this result.

Let's see if Batch Normalization will improve the performance.